In [3]:
library(readxl)
library(dplyr)


In [9]:
# Adatok beolvasása
fuel_prices <- read_excel("fuel_prices_tidy.xlsx")

In [12]:
fuel_prices <- fuel_prices %>%
  mutate(
    dummy_fovarosi_kerulet = ifelse(is_motorway == 1, 0, dummy_fovarosi_kerulet),
    dummy_kozseg = ifelse(is_motorway == 1, 0, dummy_kozseg),
    dummy_megyei_jogu_varos = ifelse(is_motorway == 1, 0, dummy_megyei_jogu_varos),
    dummy_megye_szekhely_megyei_jogu_varos = ifelse(is_motorway == 1, 0, dummy_megye_szekhely_megyei_jogu_varos),
    dummy_nagykkozseg = ifelse(is_motorway == 1, 0, dummy_nagykkozseg),
    dummy_varos = ifelse(is_motorway == 1, 0, dummy_varos),
    popper_modified = ifelse(nearest_distance_km == 0, pop_per_station, 0)
  )

In [13]:
# Csomagok
library(dplyr)

# Lineáris regresszió
model_full <- lm(
  Diesel ~ Population + Dwellings + pop_per_station + popper_modified +
    is_motorway + nearest_diff_brand_km + nearest_distance_km + income + nearest_refinery_km +
     dummy_kozseg + dummy_megyei_jogu_varos +
    dummy_megye_szekhely_megyei_jogu_varos + dummy_nagykkozseg + dummy_varos +
    dummy_Mol + dummy_Shell + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol +
    dummy_Oil + dummy_Avia + dummy_Mol_Partner + dummy_Auchan +
    dummy_Volnbusz_Zrt + dummy_Hunpetrol_2018_Kft + dummy_Edo +
    dummy_Oranges_Oil_Company + dummy_Concordia_Trans_Kft + dummy_Oplus_Kft +
    dummy_TeleTank + dummy_GROVI_Kft,
  data = fuel_prices
)

# Eredmények összefoglalása
summary(model_full)


Call:
lm(formula = Diesel ~ Population + Dwellings + pop_per_station + 
    popper_modified + is_motorway + nearest_diff_brand_km + nearest_distance_km + 
    income + nearest_refinery_km + dummy_kozseg + dummy_megyei_jogu_varos + 
    dummy_megye_szekhely_megyei_jogu_varos + dummy_nagykkozseg + 
    dummy_varos + dummy_Mol + dummy_Shell + dummy_Omv + dummy_Orlen + 
    dummy_Mobil_Petrol + dummy_Oil + dummy_Avia + dummy_Mol_Partner + 
    dummy_Auchan + dummy_Volnbusz_Zrt + dummy_Hunpetrol_2018_Kft + 
    dummy_Edo + dummy_Oranges_Oil_Company + dummy_Concordia_Trans_Kft + 
    dummy_Oplus_Kft + dummy_TeleTank + dummy_GROVI_Kft, data = fuel_prices)

Residuals:
    Min      1Q  Median      3Q     Max 
-48.027  -0.964   0.155   1.089  33.009 

Coefficients:
                                         Estimate Std. Error t value Pr(>|t|)
(Intercept)                             5.895e+02  4.180e+00 141.033  < 2e-16
Population                              1.977e-06  2.726e-05   0.073  0.94218

In [17]:
library(broom)
library(dplyr)

# Regresszió (ha már lefuttattad, ezt nem kell újra futtatni)
# model_full <- lm(...)

# Koeficiensek p-érték szerint rendezve
tidy(model_full) %>%
  filter(term != "(Intercept)") %>%        # konstans kihagyása
  arrange(desc(p.value)) %>%                     # növekvő sorrendben p-érték szerint
  mutate(p_value = round(p.value, 5)) %>%  # p-érték 5 tizedesre kerekítve
  select(term, p_value) %>%
  print(n = 30)    

# A tibble: 31 × 2
   term                                   p_value
   <chr>                                    <dbl>
 1 pop_per_station                        0.992  
 2 Population                             0.942  
 3 Dwellings                              0.840  
 4 dummy_kozseg                           0.809  
 5 popper_modified                        0.744  
 6 dummy_GROVI_Kft                        0.670  
 7 dummy_Mobil_Petrol                     0.644  
 8 nearest_refinery_km                    0.592  
 9 dummy_Oil                              0.477  
10 dummy_TeleTank                         0.376  
11 dummy_nagykkozseg                      0.360  
12 dummy_Mol_Partner                      0.359  
13 dummy_Oplus_Kft                        0.308  
14 dummy_varos                            0.299  
15 dummy_Hunpetrol_2018_Kft               0.245  
16 dummy_Oranges_Oil_Company              0.240  
17 dummy_Edo                              0.221  
18 dummy_megye_szekhely_megyei_

In [ ]:
# Dummy változók listája
dummy_vars <- c(
  "is_motorway",
  "dummy_Mol",
  "dummy_Omv",
  "dummy_Shell",
  "dummy_Auchan",
  "dummy_Orlen",
  "dummy_Volnbusz_Zrt",
  "dummy_Concordia_Trans_Kft",
  "dummy_Avia"
)

# Formula automatikusan
formula_str <- paste("Diesel ~", paste(dummy_vars, collapse = " + "))
formula <- as.formula(formula_str)

# Lineáris regresszió
m_diesel <- lm(formula, data = fuel_prices)

# Eredmények összegzése
summary(m_diesel)

In [22]:
fuel_prices <- fuel_prices %>%
  mutate(
    log_Population = log(Population),
    log_Dwellings = log(Dwellings),
    log_pop_per_station = log1p(pop_per_station),
    log_popper_modified = log1p(popper_modified),
    log_nearest_diff_brand_km = log1p(nearest_diff_brand_km),
    log_nearest_distance_km = log1p(nearest_distance_km),
    log_income = log(income),
    log_nearest_refinery_km = log(nearest_refinery_km)
  )

In [23]:
# Feltételezzük, hogy a kategórialisták már léteznek a környezetben:
# sure_useful_vars, density_vars, competition_vars, demographic_vars,
# income_vars, refinery_vars, other_vars
# (És hogy a referencia-dummyk: dummy_fovarosi_kerulet és dummy_Other nincsenek a listákban.)

library(dplyr)
library(purrr)
library(tibble)

n <- 20

# 0) Biztonsági ellenőrzés: minden változó egyszer szerepeljen
all_vars_cat <- c(sure_useful_vars, density_vars, competition_vars,
                  demographic_vars, income_vars, refinery_vars, other_vars)
dups <- all_vars_cat[duplicated(all_vars_cat)]
if (length(dups) > 0) stop("Duplikált változók a kategóriák között: ", paste(unique(dups), collapse = ", "))

# 1) Kiválasztási szabályoknak megfelelő választékok összeállítása
#    - Ezeknél: legfeljebb 1 változó → {Ø} ∪ {egyenkénti}
pick0_or_1 <- function(v) c(list(character(0)), as.list(v))

density_choices     <- pick0_or_1(density_vars)
competition_choices <- pick0_or_1(competition_vars)
demographic_choices <- pick0_or_1(demographic_vars)
income_choices      <- pick0_or_1(income_vars)
refinery_choices    <- pick0_or_1(refinery_vars)

#    - other_vars: összes részhalmaz (Ø is)
power_set <- function(v) {
  if (length(v) == 0) return(list(character(0)))
  unlist(lapply(0:length(v), function(k) combn(v, k, simplify = FALSE)), recursive = FALSE)
}
other_subsets <- power_set(other_vars)

# 2) Végigmegyünk az összes kombináción és futtatjuk a modelleket
results <- vector("list", length(density_choices) *
                            length(competition_choices) *
                            length(demographic_choices) *
                            length(income_choices) *
                            length(refinery_choices) *
                            length(other_subsets))

ix <- 1L

# Opcionális: egyszerű progress jelző
total <- length(results)
pb <- utils::txtProgressBar(min = 0, max = total, style = 3)

for (d in density_choices) {
  for (c in competition_choices) {
    for (g in demographic_choices) {
      for (inc in income_choices) {
        for (r in refinery_choices) {
          for (o in other_subsets) {

            sel_vars <- c(sure_useful_vars, d, c, g, inc, r, o)
            # Ha véletlenül üres (elvileg nem lehet, mert sure_useful mindig van), védjük:
            if (length(sel_vars) == 0) sel_vars <- "1"

            frm <- as.formula(paste("Diesel ~", paste(sel_vars, collapse = " + ")))

            fit <- try(lm(frm, data = fuel_prices), silent = TRUE)
            if (inherits(fit, "try-error")) {
              # Ha valamiért elhasal egy kombináció, jelöljük NA-val (ritka)
              results[[ix]] <- tibble(
                BIC = NA_real_, AIC = NA_real_, n_vars = length(sel_vars),
                density_pick     = ifelse(length(d)==0, "(none)", paste(d, collapse="+")),
                competition_pick = ifelse(length(c)==0, "(none)", paste(c, collapse="+")),
                demographic_pick = ifelse(length(g)==0, "(none)", paste(g, collapse="+")),
                income_pick      = ifelse(length(inc)==0, "(none)", paste(inc, collapse="+")),
                refinery_pick    = ifelse(length(r)==0, "(none)", paste(r, collapse="+")),
                other_pick       = ifelse(length(o)==0, "(none)", paste(o, collapse="+")),
                formula_str      = paste("Diesel ~", paste(sel_vars, collapse = " + "))
              )
            } else {
              results[[ix]] <- tibble(
                BIC = BIC(fit),
                AIC = AIC(fit),
                n_vars = length(sel_vars),
                density_pick     = ifelse(length(d)==0, "(none)", paste(d, collapse="+")),
                competition_pick = ifelse(length(c)==0, "(none)", paste(c, collapse="+")),
                demographic_pick = ifelse(length(g)==0, "(none)", paste(g, collapse="+")),
                income_pick      = ifelse(length(inc)==0, "(none)", paste(inc, collapse="+")),
                refinery_pick    = ifelse(length(r)==0, "(none)", paste(r, collapse="+")),
                other_pick       = ifelse(length(o)==0, "(none)", paste(o, collapse="+")),
                formula_str      = paste("Diesel ~", paste(sel_vars, collapse = " + "))
              )
            }

            ix <- ix + 1L
            if ((ix - 1L) %% 250 == 0) utils::setTxtProgressBar(pb, ix - 1L)
          }
        }
      }
    }
  }
}
close(pb)

# 3) Eredmények összefűzése és Top-15 BIC szerint
res_df <- bind_rows(results)

# Ha bárhol NA lett (ritka), dobjuk:
res_df <- res_df %>% filter(!is.na(BIC))

top15 <- res_df %>%
  arrange(AIC, BIC, n_vars) %>%   # elsődlegesen BIC, majd AIC, végül kevesebb változó előnyben
  slice(1:n) %>%
  select(
    AIC, BIC, n_vars,
    density_pick, competition_pick, demographic_pick,
    income_pick, refinery_pick, other_pick,
    formula_str
  )

# 4) Top-15 megjelenítése
print(top15, n = n)

  |======================================================================| 100%
# A tibble: 20 × 10
     AIC   BIC n_vars density_pick competition_pick demographic_pick income_pick
   <dbl> <dbl>  <int> <chr>        <chr>            <chr>            <chr>      
 1 7615. 7691.     13 log_popper_… log_nearest_dif… Population       (none)     
 2 7615. 7681.     11 (none)       log_nearest_dis… log_Population   (none)     
 3 7615. 7686.     12 (none)       log_nearest_dis… log_Population   (none)     
 4 7615. 7686.     12 log_popper_… log_nearest_dif… Population       (none)     
 5 7615. 7691.     13 log_popper_… log_nearest_dif… (none)           (none)     
 6 7615. 7696.     14 log_popper_… log_nearest_dif… Population       (none)     
 7 7615. 7681.     11 (none)       log_nearest_dis… log_Dwellings    (none)     
 8 7615. 7691.     13 log_popper_… log_nearest_dif… Dwellings        (none)     
 9 7615. 7686.     12 (none)       log_nearest_dis… log_Dwellings    (none)     
10 7615. 

In [26]:
# Dummy változók listája
dummy_vars <- c(
  "is_motorway",
  "dummy_Mol",
  "dummy_Omv",
  "dummy_Shell",
  "dummy_Auchan",
  "dummy_Orlen",
  "dummy_Volnbusz_Zrt",
  "dummy_Concordia_Trans_Kft",
  "dummy_Avia"
)

# Formula automatikusan
formula_str <- paste("Diesel ~", paste(dummy_vars, collapse = " + "))
formula <- as.formula(formula_str)

# Lineáris regresszió
m_diesel <- lm(formula, data = fuel_prices)

# Eredmények összegzése
summary(m_diesel)



Call:
lm(formula = formula, data = fuel_prices)

Residuals:
    Min      1Q  Median      3Q     Max 
-49.174  -0.649   0.000   0.662  35.219 

Coefficients:
                          Estimate Std. Error  t value Pr(>|t|)    
(Intercept)               583.4956     0.3721 1567.914  < 2e-16 ***
is_motorway                41.7211     0.6753   61.781  < 2e-16 ***
dummy_Mol                   7.2965     0.4752   15.354  < 2e-16 ***
dummy_Omv                   6.9831     0.5829   11.979  < 2e-16 ***
dummy_Shell                 6.3140     0.5720   11.039  < 2e-16 ***
dummy_Auchan              -12.4956     1.3980   -8.938  < 2e-16 ***
dummy_Orlen                 5.7244     0.6226    9.195  < 2e-16 ***
dummy_Volnbusz_Zrt         -7.2711     2.2510   -3.230  0.00127 ** 
dummy_Concordia_Trans_Kft   4.5044     2.4266    1.856  0.06367 .  
dummy_Avia                  2.5701     1.2554    2.047  0.04086 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 5

In [ ]:
# 1. kategória: biztosan hasznos változók
sure_useful_vars <- c(
  "is_motorway",
  "dummy_Mol",
  "dummy_Shell",
  "dummy_Auchan",
  "dummy_Omv",
  "dummy_Orlen",
  "dummy_Mobil_Petrol",
  "dummy_nagykkozseg",
  "dummy_kozseg"
)

# 2. kategória: népsűrűséghez / kutasűrűséghez kapcsolódó változók
density_vars <- c(
  "pop_per_station",
  "popper_modified",
  "log_pop_per_station",
  "log_popper_modified"
)

# 3. kategória: versenytávolság / térbeli elhelyezkedés változói
competition_vars <- c(
  "log_nearest_diff_brand_km",
  "log_nearest_distance_km",
  "nearest_diff_brand_km",
  "nearest_distance_km"
)

# 4. kategória: demográfiai változók
demographic_vars <- c(
  "Population",
  "Dwellings",
  "log_Population",
  "log_Dwellings"
)

# 5. kategória: jövedelmi változók
income_vars <- c(
  "income",
  "log_income"
)

# 6. kategória: haszontalan / nem szignifikáns változók
useless_vars <- c(
  "dummy_Concordia_Trans_Kft",
  "dummy_GROVI_Kft",
  "dummy_Oil",
  "dummy_Mol_Partner",
  "dummy_megye_szekhely_megyei_jogu_varos",
  "dummy_megyei_jogu_varos",
  "dummy_TeleTank",
  "dummy_Mobil_Petrol"
)

# 7. kategória: finomító közelsége / logisztikai tényezők
refinery_vars <- c(
  "nearest_refinery_km",
  "log_nearest_refinery_km"
)

# 7. kategória: egyéb változók
other_vars <- c(
  "dummy_Edo",
  "dummy_Oplus_Kft",
  "dummy_varos",
  "dummy_Oranges_Oil_Company",
  "dummy_Hunpetrol_2018_Kft"
)


In [24]:
library(rsample)
library(dplyr)
library(purrr)
library(tibble)
library(yardstick)

# Feltételezzük: res_df már létezik és tartalmazza a 36k modell BIC/AIC + formula_str mezőit

# Top 50 BIC szerint
top50 <- res_df %>%
  arrange(AIC) %>%
  slice(1:50)

# 5-fold CV a TELJES adathalmazon (nincs külön train/test)
set.seed(120)
cv5 <- vfold_cv(fuel_prices, v = 5)

# Segédfüggvény: átlagos out-of-sample R^2 egy formulára 5-fold CV-ben
cv_r2 <- function(formula_str, cv_obj, data) {
  fold_r2 <- map_dbl(cv_obj$splits, function(s) {
    train_fold <- analysis(s)
    valid_fold <- assessment(s)

    fit <- lm(as.formula(formula_str), data = train_fold)
    preds <- predict(fit, newdata = valid_fold)
    yardstick::rsq_trad_vec(truth = valid_fold$Diesel, estimate = preds)
  })
  mean(fold_r2)
}

# Futtatás az 50 legjobb modellre
cv_results <- map_dfr(seq_len(nrow(top50)), function(i) {
  fstr <- top50$formula_str[i]
  r2cv <- cv_r2(fstr, cv5, fuel_prices)
  tibble(
    model_rank_BIC = i,
    mean_cv_r2 = r2cv,
    BIC = top50$BIC[i],
    AIC = top50$AIC[i],
    n_vars = top50$n_vars[i],
    density_pick     = top50$density_pick[i],
    competition_pick = top50$competition_pick[i],
    demographic_pick = top50$demographic_pick[i],
    income_pick      = top50$income_pick[i],
    refinery_pick    = top50$refinery_pick[i],
    other_pick       = top50$other_pick[i],
    formula_str      = fstr
  )
})

# 10 legjobb modell out-of-sample R^2 alapján
top10_cv <- cv_results %>%
  arrange(desc(mean_cv_r2), BIC, AIC, n_vars) %>%
  slice(1:10)

print(top10_cv, n = 10)


Warning message:
"a(z) 'yardstick' csomag az R 4.4.3 verziójával lett fordítva"


# A tibble: 10 × 12
   model_rank_BIC mean_cv_r2   BIC   AIC n_vars density_pick    competition_pick
            <int>      <dbl> <dbl> <dbl>  <int> <chr>           <chr>           
 1             26      0.787 7697. 7615.     14 log_popper_mod… log_nearest_dif…
 2             25      0.787 7692. 7615.     13 (none)          log_nearest_dis…
 3             45      0.787 7692. 7616.     13 (none)          log_nearest_dis…
 4             48      0.787 7697. 7616.     14 log_popper_mod… log_nearest_dif…
 5              6      0.787 7696. 7615.     14 log_popper_mod… log_nearest_dif…
 6             17      0.787 7697. 7615.     14 log_popper_mod… log_nearest_dif…
 7             47      0.787 7692. 7616.     13 (none)          log_nearest_dis…
 8             39      0.787 7697. 7616.     14 log_popper_mod… log_nearest_dif…
 9             16      0.787 7691. 7615.     13 (none)          log_nearest_dis…
10             10      0.787 7696. 7615.     14 log_popper_mod… log_nearest_dif…
# ℹ 5 mo

In [25]:
top10_cv

model_rank_BIC,mean_cv_r2,BIC,AIC,n_vars,density_pick,competition_pick,demographic_pick,income_pick,refinery_pick,other_pick,formula_str
<int>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
26,0.7866244,7696.734,7615.413,14,log_popper_modified,log_nearest_diff_brand_km,Population,(none),(none),dummy_Edo+dummy_Oranges_Oil_Company,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_popper_modified + log_nearest_diff_brand_km + Population + dummy_Edo + dummy_Oranges_Oil_Company
25,0.7866100,7691.647,7615.409,13,(none),log_nearest_distance_km,Population,(none),(none),dummy_Edo+dummy_Hunpetrol_2018_Kft,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_nearest_distance_km + Population + dummy_Edo + dummy_Hunpetrol_2018_Kft
45,0.7865962,7691.881,7615.643,13,(none),log_nearest_distance_km,Dwellings,(none),(none),dummy_Edo+dummy_Hunpetrol_2018_Kft,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_nearest_distance_km + Dwellings + dummy_Edo + dummy_Hunpetrol_2018_Kft
48,0.7865934,7697.011,7615.690,14,log_popper_modified,log_nearest_diff_brand_km,Dwellings,(none),(none),dummy_Edo+dummy_Oranges_Oil_Company,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_popper_modified + log_nearest_diff_brand_km + Dwellings + dummy_Edo + dummy_Oranges_Oil_Company
6,0.7865853,7696.214,7614.894,14,log_popper_modified,log_nearest_diff_brand_km,Population,(none),(none),dummy_Edo+dummy_Hunpetrol_2018_Kft,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_popper_modified + log_nearest_diff_brand_km + Population + dummy_Edo + dummy_Hunpetrol_2018_Kft
17,0.7865371,7696.533,7615.212,14,log_popper_modified,log_nearest_diff_brand_km,Dwellings,(none),(none),dummy_Edo+dummy_Hunpetrol_2018_Kft,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_popper_modified + log_nearest_diff_brand_km + Dwellings + dummy_Edo + dummy_Hunpetrol_2018_Kft
47,0.7865365,7691.905,7615.666,13,(none),log_nearest_distance_km,log_Dwellings,(none),(none),dummy_Edo+dummy_Oranges_Oil_Company,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_nearest_distance_km + log_Dwellings + dummy_Edo + dummy_Oranges_Oil_Company
39,0.7865350,7696.871,7615.550,14,log_popper_modified,log_nearest_diff_brand_km,(none),(none),(none),dummy_Edo+dummy_varos+dummy_Oranges_Oil_Company,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_popper_modified + log_nearest_diff_brand_km + dummy_Edo + dummy_varos + dummy_Oranges_Oil_Company
16,0.7865251,7691.447,7615.209,13,(none),log_nearest_distance_km,log_Dwellings,(none),(none),dummy_Edo+dummy_Hunpetrol_2018_Kft,Diesel ~ is_motorway + dummy_Mol + dummy_Shell + dummy_Auchan + dummy_Omv + dummy_Orlen + dummy_Mobil_Petrol + dummy_nagykkozseg + dummy_kozseg + log_nearest_distance_km + log_Dwellings + dummy_Edo + dummy_Hunpetrol_2018_Kft
